## 1. API 받아오기

In [1]:
import requests

url = "https://api.upbit.com/v1/candles/minutes/1?market=KRW-BTC&count=200"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

print(response.text)
print(type(response.text))

[{"market":"KRW-BTC","candle_date_time_utc":"2023-09-01T06:23:00","candle_date_time_kst":"2023-09-01T15:23:00","opening_price":35374000.00000000,"high_price":35376000.00000000,"low_price":35368000.00000000,"trade_price":35376000.00000000,"timestamp":1693549391343,"candle_acc_trade_price":33914326.77822000,"candle_acc_trade_volume":0.95885420,"unit":1},{"market":"KRW-BTC","candle_date_time_utc":"2023-09-01T06:22:00","candle_date_time_kst":"2023-09-01T15:22:00","opening_price":35377000.00000000,"high_price":35377000.00000000,"low_price":35367000.00000000,"trade_price":35375000.00000000,"timestamp":1693549377815,"candle_acc_trade_price":119733177.91945000,"candle_acc_trade_volume":3.38521109,"unit":1},{"market":"KRW-BTC","candle_date_time_utc":"2023-09-01T06:21:00","candle_date_time_kst":"2023-09-01T15:21:00","opening_price":35387000.00000000,"high_price":35387000.00000000,"low_price":35368000.00000000,"trade_price":35377000.00000000,"timestamp":1693549319811,"candle_acc_trade_price":1321

## 2. DataFrame으로 변환하여 csv로 저장

In [10]:
import pandas as pd
import json

tmp = open("tmp.txt", "w")
tmp.write(response.text)
tmp.close()

with open("tmp.txt") as f:
    _tmp = json.loads(f.read())

df = pd.DataFrame(_tmp)
df.to_csv("chart.csv")

df = pd.read_csv("chart.csv")

print(df)

     Unnamed: 0   market candle_date_time_utc candle_date_time_kst  \
0             0  KRW-BTC  2023-08-30T17:10:00  2023-08-31T02:10:00   
1             1  KRW-BTC  2023-08-30T17:09:00  2023-08-31T02:09:00   
2             2  KRW-BTC  2023-08-30T17:08:00  2023-08-31T02:08:00   
3             3  KRW-BTC  2023-08-30T17:07:00  2023-08-31T02:07:00   
4             4  KRW-BTC  2023-08-30T17:06:00  2023-08-31T02:06:00   
..          ...      ...                  ...                  ...   
195         195  KRW-BTC  2023-08-30T13:55:00  2023-08-30T22:55:00   
196         196  KRW-BTC  2023-08-30T13:54:00  2023-08-30T22:54:00   
197         197  KRW-BTC  2023-08-30T13:53:00  2023-08-30T22:53:00   
198         198  KRW-BTC  2023-08-30T13:52:00  2023-08-30T22:52:00   
199         199  KRW-BTC  2023-08-30T13:51:00  2023-08-30T22:51:00   

     opening_price  high_price   low_price  trade_price      timestamp  \
0       36867000.0  36889000.0  36867000.0   36887000.0  1693415418915   
1       368

## 3.MySQL과 연동하기

In [13]:
import pandas as pd 
import pymysql

conn = pymysql.connect(host='127.0.0.1', user = 'root', password='1133', db = 'upbit',charset = 'utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)

## 4. 파이썬 코드로 Table 생성하기

In [18]:
sql = '''create table chart(
idx int,
market VARCHAR(10),
candle_date_time_utc VARCHAR(30),
candle_date_time_kst VARCHAR(30),
opening_price double,
high_price double,
low_price double,
trade_price	double,
timestamp long,
candle_acc_trade_price double,
candle_acc_trade_volume	double,
unit int);'''
curs.execute(sql)
conn.commit()

## 5. Table에 데이터 넣기

In [60]:
sql = '''insert into chart
(idx, market, candle_date_time_utc, candle_date_time_kst, opening_price, high_price,
low_price, trade_price, timestamp, candle_acc_trade_price, candle_acc_trade_volume, unit)
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

for idx in range(len(df)):
	curs.execute(sql, tuple(df.values[idx]))
conn.commit()

## 6. 파이썬과 연동 종료

In [62]:
#종료
curs.close()
conn.close()